In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_scaled = StandardScaler().fit_transform(df_market_data[['price_change_percentage_24h', 'price_change_percentage_7d', 'price_change_percentage_14d',
                                                                    'price_change_percentage_30d', 'price_change_percentage_60d', 'price_change_percentage_200d',
                                                                    'price_change_percentage_1y']])

market_data_scaled[0:5]

In [ ]:
# Create a DataFrame with the scaled data
market_data_df = pd.DataFrame(market_data_scaled, columns = ['price_change_percentage_24h', 'price_change_percentage_7d', 'price_change_percentage_14d',
                                                                    'price_change_percentage_30d', 'price_change_percentage_60d', 'price_change_percentage_200d',
                                                                    'price_change_percentage_1y'])

# Copy the crypto names from the original data
coinid = df_market_data.index.copy()
# Set the coinid column as index
market_data_df2 = market_data_df.set_index(coinid)

# Display sample data


In [ ]:
market_data_df2.head(5)

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
inertia = []
k = list(range(1, 11))

In [ ]:
# Create an empty list to store the inertia values

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(market_data_df2)
    inertia.append(k_model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"K":k, "inertia":inertia}

elbow_data


# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

df_elbow.head()


In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
market_elbow = df_elbow.hvplot.line(x ='K', y= 'inertia', title = " Market Elbow Curve", xticks = k)

market_elbow


#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [ ]:
# Fit the K-Means model using the scaled data
model.fit(market_data_df2)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
predict_model = model.predict(market_data_df2)

# Print the resulting array of cluster values.
print(predict_model)

In [ ]:
# Create a copy of the DataFrame
market_data_predictions_df = market_data_df2.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
market_data_predictions_df['Crypto Clusters'] = predict_model

# Display sample data
market_data_predictions_df.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

market_predict_plot = market_data_predictions_df.hvplot.scatter(
    x= 'price_change_percentage_24h',
    y= 'price_change_percentage_7d',
    by= 'Crypto Clusters',
    hover_cols= 'coin_id'
)

market_predict_plot




---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_data_pca = pca.fit_transform(market_data_df2)

# View the first five rows of the DataFrame. 
market_data_pca[:5]

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca_array = pca.explained_variance_ratio_
sumpca = sum(pca_array)

print(pca_array)
print(f"{sumpca:.2%}")

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** About 89.5% of the total variance is condensed into the three principal components.

In [ ]:
# Create a new DataFrame with the PCA data.

# Creating a DataFrame with the PCA data
market_data_pca_df = pd.DataFrame(market_data_pca, columns = ['PC1', 'PC2', 'PC3'])

# Copy the crypto names from the original data
coinid = df_market_data.index.copy()

# Set the coinid column as index
market_data_pca_df = market_data_pca_df.set_index(coinid)

# Display sample data
market_data_pca_df.head(5)


---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
inertia = []
k = list(range(1, 11))

In [ ]:
# Create an empy list to store the inertia values


# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(market_data_pca_df)
    inertia.append(k_model.inertia_)


In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {"K":k, "inertia":inertia}


# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data_pca)

df_elbow.head()


In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_pca = df_elbow.hvplot.line(x ='K', y= 'inertia', title = "Elbow Curve for PCA", xticks = k)

elbow_pca

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No it does not.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [ ]:
# Fit the K-Means model using the PCA data
model.fit(market_data_pca_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
predict_model_pca = model.predict(market_data_pca_df)

# Print the resulting array of cluster values.
print(predict_model_pca)

In [ ]:
# Create a copy of the DataFrame with the PCA data
market_pca_predictions_df = market_data_pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters

market_pca_predictions_df['Market Clusters'] = predict_model_pca


# Display sample data
market_pca_predictions_df.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# x="PC1" and `y="PC2". 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

market_pca_plot = market_pca_predictions_df.hvplot.scatter(
    x= 'PC1',
    y= 'PC2',
    by= 'Market Clusters',
    hover_cols= 'coin_id'
)

market_pca_plot

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves


market_elbow + elbow_pca

In [ ]:
# Composite plot to contrast the clusters
market_predict_plot + market_pca_plot

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The impact of utilizing a principal component analysis is that by reducing the dimensions tested, the algorithm makes the data simplier to explore.  Our first K-Means clusters plotted are not highly segregated.  The inflection point on both elbow plots are at the same point, determining 4 clusters is our optimal cluster count.  The PCA plot reduces the dimensions of our data removing outliers from our plot, making it clearer to distinguish the data.